In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from pickle import load

In [2]:
# Load Data
customers = pd.read_csv('../data/customers.csv')
articles = pd.read_csv('../data/articles.csv')
transactions = pd.read_csv('../data/transactions_train.csv')

In [86]:
# KNN recommd function
def recom_knn(art_ID):

    query_artcile = art_ID

    file = "../data/newKnn.pickle"
    K = load(open(file,"rb"))
    
    ALL_USERS = customers['customer_id'].unique().tolist()
    ALL_ITEMS = articles['article_id'].unique().tolist()

    leUser = LabelEncoder().fit(ALL_USERS)
    transactions['user_id'] = leUser.transform(transactions['customer_id'])
    leItem = LabelEncoder().fit(ALL_ITEMS)
    transactions['item_id'] = leItem.transform(transactions['article_id'])

    col = transactions['user_id'].values
    row = transactions['item_id'].values
    one = np.ones(transactions.shape[0])
    coo = coo_matrix((one, (row, col)), shape=(len(ALL_ITEMS), len(ALL_USERS)))
    csr = coo.tocsr()

    model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
    model_knn.fit(csr)
    index = leItem.transform([query_artcile])
    distances, indices = K.kneighbors(csr[index,:].toarray().reshape(1,-1), n_neighbors = 6)
    
    article = []
    distance = []

    for i in range(0, len(distances.flatten())):
        if i != 0:
            article.append(leItem.inverse_transform([indices.flatten()[i]]))
            distance.append(distances.flatten()[i]) 
           

    m=pd.Series(article,name='article')
    d=pd.Series(distance,name='distance')
    recommend = pd.concat([m,d], axis=1)
    recommend = recommend.sort_values('distance',ascending=False)

    # for i in range(0,recommend.shape[0]):
        # recom_list.append(recommend["article"].iloc[i])
        # recommend["article"].iloc[i]
    recom_list = [recommend["article"].iloc[i] for i in range(0,recommend.shape[0])]

    
    return recom_list


In [87]:
rk = recom_knn(541518023)

In [89]:
for i in rk:
    print(i.item())

708346001
572546008
597173002
318035015
649311001
